This computes the weighted slope by combining OLS(x,y), the reciprocal of OLS(y,x), and TLS(x,y), along with their respective weighted errors.

In [1]:
import numpy as np
import math
import pandas as pd
from scipy.odr import Model, RealData, ODR
from scipy.optimize import curve_fit
from scipy import stats



In [2]:
def linear_regression(x, y, dx, dy):
    # TLS method
    linear_tls = Model(lambda B, a: B[0]*a)
    data_tls = RealData(x, y, sx=dx, sy=dy)
    odr_tls = ODR(data_tls, linear_tls, beta0=[1.0])
    output_tls = odr_tls.run()
    slope_tls = output_tls.beta
    err_tls = output_tls.sd_beta

    # OLS method
    degrees_x = [1]  # list of degrees of x to use
    matrix_x = np.stack([x**d for d in degrees_x], axis=-1)
    slope_ols_x, _, _, _ = np.linalg.lstsq(matrix_x, y, rcond=None)
    n_x = len(y)
    k_x = len(slope_ols_x)
    sigma2_x = np.sum((y - np.dot(matrix_x, slope_ols_x))**2) / (n_x - k_x)  # RMSE
    a_x = matrix_x.T
    C_x = sigma2_x * np.linalg.inv(np.dot(a_x, matrix_x))  # covariance matrix
    err_ols_x = np.sqrt(np.diag(C_x))  # standard error

    degrees_y = [1]  # list of degrees of y to use
    matrix_y = np.stack([y**d for d in degrees_y], axis=-1)
    slope_ols_y, _, _, _ = np.linalg.lstsq(matrix_y, x, rcond=None)
    n_y = len(x)
    k_y = len(slope_ols_y)
    sigma2_y = np.sum((x - np.dot(matrix_y, slope_ols_y))**2) / (n_y - k_y)  # RMSE
    a_y = matrix_y.T
    C_y = sigma2_y * np.linalg.inv(np.dot(a_y, matrix_y))  # covariance matrix
    err_ols_y = np.sqrt(np.diag(C_y))  # standard error

    dn = 1 / (err_tls**2) + 1 / (err_ols_x**2) + 1 / (err_ols_y**2) #denominator
    slope = (slope_tls / err_tls**2 + slope_ols_x / err_ols_x**2 + 1 / slope_ols_y / err_ols_y**2) / dn
    error = math.sqrt((abs(slope_tls - slope) / err_tls**2 + abs(slope_ols_x - slope) / err_ols_x**2 + abs(1 / slope_ols_y - slope) / err_ols_y**2) / dn)

    return slope, error

# Example usage:
# slope, d_op = linear_regression(x, y, dx, dy)
# print(f"Slope: {slope}, d_op: {d_op}")


In [3]:
df = pd.read_csv(r'C:\Users\ShreyaB\Documents\Mitteilungen\data\weighted_average_example.csv')

In [4]:
df.head()

,Date,col1,col2
0,1854-02-04,3,3
1,1854-03-17,2,2
2,1854-09-12,3,3
3,1855-10-20,2,2
4,1857-01-14,3,3


In [5]:
x= df.col1
y= df.col2

In [6]:
df["dx"] = x.std()
df["dy"] = y.std()

In [7]:
slope, error = linear_regression(x, y, df.dx, df.dy)

In [8]:
slope, error

(array([0.98807214]), 0.03177069306498258)